In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from open_CLIP_experiment import *

ModuleNotFoundError: No module named 'open_CLIP_experiment'

In [5]:
for preposition_file in static_prepositions[:1]:
    # print(f"Processing {preposition_file}")
    csv_file = annotations_path / preposition_file
    image_paths, candidate_labels, target_labels = process_csv_file(csv_file, dataset_parent_path)
    

In [42]:
#iterate image_paths, candidate_labels and target_labels in batches of 32
for i in range(0, len(image_paths), 32):
    image_batch = image_paths[i:i+32]
    candidate_labels_batch = candidate_labels[i:i+32]
    target_labels_batch = target_labels[i:i+32]
    
    #convert to tensors
    image_batch = torch.stack([preprocess(Image.open(image_path)) for image_path in image_batch]).to(device)
    candidate_labels_batch = torch.stack([tokenizer(candidates) for candidates in candidate_labels_batch]).to(device)
    target_labels_batch = torch.cat([tokenizer(target_labels) for target_labels in target_labels_batch]).to(device)

    break


In [43]:
image_batch.shape, candidate_labels_batch.shape, target_labels_batch.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32, 30, 77]), torch.Size([32, 77]))

In [14]:
candidate_labels_batch[1].shape

torch.Size([30, 77])

In [45]:
out = model(image_batch,target_labels_batch)
a,b,c = out
a.shape, b.shape, c.shape

torch.Size([32, 512])


(torch.Size([32, 512]), torch.Size([32, 512]), torch.Size([]))

In [46]:
loss(a,b,c, output_dict = True)

{'contrastive_loss': tensor(5.5460, device='cuda:0', grad_fn=<DivBackward0>)}

(torch.Size([1, 30]), torch.Size([30, 1]))

In [24]:
labels = loss.get_ground_truth(device, logits_per_image.shape[0])

total_loss = (
        F.cross_entropy(a, labels) +
        F.cross_entropy(b, labels)
    ) / 2

In [31]:
loss(**out, output_dict = True)

TypeError: ClipLoss() argument after ** must be a mapping, not tuple

In [47]:
get_dataloader(all_prepositions, annotations_path, dataset_parent_path, batch_size=32, num_workers=4)

NameError: name 'get_dataloader' is not defined